In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
import json

import sys
sys.path.insert(0, '../ToolBox')

import CrossValidation, MLP, Helper, ZScore, RandomForest

import os, sys
import cPickle

In [2]:
saved_train_data = open('pre_processed_data/train_preprocessed.pkl', 'rb')
normalized_train_data = cPickle.load(saved_train_data)

In [3]:
saved_test_data = open('pre_processed_data/test_preprocessed.pkl', 'rb')
normalized_test_data = cPickle.load(saved_test_data)

In [4]:
import yaml

with open("../ToolBox/params.json") as data_file:
    params = yaml.safe_load(data_file)
print 'json loaded'
labels = normalized_train_data[:,0]
print labels.shape
labels = np.reshape(labels, [labels.shape[0], 1])
print labels.shape
labels = np.subtract(np.array(labels), 1)
data_dict = dict({'x':normalized_train_data[:,1:], 'y':Helper.convert_one_hot_encoding(labels,params['MLPandRF']['MLP']['output-dim'])})

print data_dict['y'].shape

mlp = MLP.init(params['MLPandRF']['MLP'])
mlp['train'](data_dict)

data_dict['x'] = mlp['predict'](data_dict['x'], 1)
data_dict['y'] = labels

print np.array(data_dict['x']).shape

test_transformed = mlp['predict'](normalized_test_data, 1)

rf = RandomForest.init(params['MLPandRF']['RF'])
model = rf['train'](data_dict)
result_dict = rf['predict_probabilities'](test_transformed, model)
result_dict

json loaded
(26729,)
(26729, 1)
(26729, 5)
(26729, 10)


../ToolBox/RandomForest.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(data['x'], data['y'])


array([[ 0.03356976,  0.00273399,  0.1074438 ,  0.33948071,  0.51677174],
       [ 0.53675905,  0.0016309 ,  0.02560891,  0.26244556,  0.17355558],
       [ 0.36152865,  0.0027082 ,  0.02979702,  0.1216442 ,  0.48432193],
       ..., 
       [ 0.01227349,  0.00696045,  0.06599601,  0.01135801,  0.90341205],
       [ 0.32829691,  0.00125175,  0.05301526,  0.43592707,  0.18150902],
       [ 0.02273914,  0.00588772,  0.23454858,  0.40186644,  0.33495812]])

In [5]:
print result_dict.shape

lis = []
lis.append("ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer")

for idx, z in enumerate(result_dict):
    string = str(idx+1)
    for prob in z:
        string += ',' + str(prob)
    lis.append(string)

lis = np.array(lis)
np.savetxt('result_shelter_animal_outcomes.csv', lis, fmt='%s', delimiter=',')

(11456, 5)
